In [1]:
import seaborn as sns

import yfinance as yf
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import ta 
import plotly.graph_objects as go
import stock_lib as stlib

from scipy.signal import find_peaks

In [2]:
data = yf.download("TSLA", period='60d', interval='5m')

macd = ta.trend.MACD(close=data['Close'])
bb = ta.volatility.BollingerBands(close=data['Close'])
cmf = ta.volume.ChaikinMoneyFlowIndicator(high=data['High'], low=data['Low'], close=data['Close'], volume=data['Volume'])
rsi = ta.momentum.RSIIndicator(close=data['Close'])

data['rsi'] = rsi.rsi()
data['macd'] = macd.macd_diff()
data['cmf'] = cmf.chaikin_money_flow()
data['bb_width'] = bb.bollinger_wband()

data['close_3'] = data['Close'].diff(periods=-5)
data['close_3'] = (data['close_3'] / data['Close']) * 100

data['close_8'] = data['Close'].diff(periods=-8)
data['close_8'] = (data['close_8'] / data['Close']) * 100

data['close_13'] = data['Close'].diff(periods=-13)
data['close_13'] = (data['close_13'] / data['Close']) * 100
data['close_21'] = data['Close'].diff(periods=-21)
data['close_21'] = (data['close_21'] / data['Close']) * 100
data = data.dropna()
data = data.reset_index()

# data['macd'] = macd.macd_diff()
# data['cmf'] = cmf.chaikin_money_flow()
# data['bb_width'] = bb.bollinger_wband()

data = data.drop(['High', 'Low', 'Adj Close'], axis=1)

data.head(10)

[*********************100%***********************]  1 of 1 completed


,Datetime,Open,Close,Volume,rsi,macd,cmf,bb_width,close_3,close_8,close_13,close_21
0,2021-06-24 12:15:00-04:00,695.830017,696.669983,538974,72.789165,-0.177410,0.221228,2.092604,0.200951,1.001188,1.223042,1.801425
1,2021-06-24 12:20:00-04:00,696.609985,695.954590,365853,69.753067,-0.215517,0.197262,2.028301,0.568225,0.550264,1.038805,1.967740
2,2021-06-24 12:25:00-04:00,695.952209,695.945007,355254,69.711123,-0.266054,0.186139,1.880235,0.683244,0.972035,1.232139,1.361463
3,2021-06-24 12:30:00-04:00,696.000000,694.588379,388197,63.856753,-0.407092,0.168855,1.627332,0.704499,0.650897,1.351636,1.167684
4,2021-06-24 12:35:00-04:00,694.630005,692.640015,478368,56.515958,-0.634296,0.164395,1.350274,0.074355,0.870586,1.361457,0.811388
5,2021-06-24 12:40:00-04:00,692.739990,695.270020,426504,62.742183,-0.609970,0.153149,1.194459,0.875896,1.024150,2.271786,1.073687
6,2021-06-24 12:45:00-04:00,695.255005,692.000000,326987,52.648311,-0.804401,0.097686,1.106788,0.279289,0.473269,1.510550,0.578255
7,2021-06-24 12:50:00-04:00,691.979980,691.190002,502395,50.481847,-0.966926,0.093244,0.962297,0.662628,0.552671,1.012764,0.504925
8,2021-06-24 12:55:00-04:00,691.190125,689.695007,666680,46.665047,-1.142445,0.031773,1.058561,0.224098,0.651728,0.808330,0.292160
9,2021-06-24 13:00:00-04:00,689.640015,692.125000,400734,52.898755,-1.063818,0.024917,1.054786,0.491244,1.288059,1.425319,0.898677


In [3]:
def regular_bearish(close_peaks, ind_peaks):
    sum1 = 0
    sum2 = 0
    if len(close_peaks) > 0:
        sum1 = close_peaks[0] - close_peaks[-1]
    if len(ind_peaks) > 0:
        sum2 = ind_peaks[0] - ind_peaks[-1]
    
    if sum1 > 0 and sum2 < 0:
        return 1
    return 0

def hidden_bearish(close_peaks, ind_peaks):
    sum1 = 0
    sum2 = 0
    if len(close_peaks) > 0:
        sum1 = close_peaks[0] - close_peaks[-1]
    if len(ind_peaks) > 0:
        sum2 = ind_peaks[0] - ind_peaks[-1]
    
    if sum1 < 0 and sum2 > 0:
        return 1
    return 0

In [4]:
label = []

for i in range(0, len(data)):
    num1 = i
    num2 = i + 20
    
    close_data = data.iloc[num1:num2][['Close']].to_numpy()
    close_data = np.array(close_data).flatten()

    index1, _ = find_peaks(close_data, width=1)
    close_peaks = close_data[index1]

    rsi_data = data.iloc[num1:num2][['cmf']].to_numpy()
    rsi_data = np.array(rsi_data).flatten()

    index2, _ = find_peaks(rsi_data, width=1)
    rsi_peaks = rsi_data[index2]
    
    label.append(regular_bearish(close_peaks, rsi_peaks))
    
data['regular_bearish'] = label

label = []

for i in range(0, len(data)):
    num1 = i
    num2 = i + 20
    
    close_data = data.iloc[num1:num2][['Close']].to_numpy()
    close_data = np.array(close_data).flatten()

    index1, _ = find_peaks(close_data, width=1)
    close_peaks = close_data[index1]

    rsi_data = data.iloc[num1:num2][['cmf']].to_numpy()
    rsi_data = np.array(rsi_data).flatten()

    index2, _ = find_peaks(rsi_data, width=1)
    rsi_peaks = rsi_data[index2]
    
    label.append(hidden_bearish(close_peaks, rsi_peaks))
    
data['hidden_bearish'] = label

In [5]:
# num1 = 20
# num2 = 40

# close_data = data.iloc[num1:num2][['Close']].to_numpy()
# close_data = np.array(close_data).flatten()

# index1, _ = find_peaks(close_data, width=1)
# close_peaks = close_data[index1]

# rsi_data = data.iloc[num1:num2][['cmf']].to_numpy()
# rsi_data = np.array(rsi_data).flatten()

# index2, _ = find_peaks(rsi_data, width=1)
# rsi_peaks = rsi_data[index2]

# print(regular_bearish(close_peaks, rsi_peaks))
# print(hidden_bearish(close_peaks, rsi_peaks))

In [6]:
# plt.plot(close_data)
# plt.plot(index1, close_data[index1], 'x')

In [7]:
# plt.plot(rsi_data)
# plt.plot(index2, rsi_data[index2], 'x')

In [8]:
data1 = data[(data['regular_bearish'] == 1) | ((data['hidden_bearish'] == 1))]

In [9]:
data = data.drop(['Datetime', 'Open', 'Close', 'Volume'], axis=1)
data

,rsi,macd,cmf,bb_width,close_3,close_8,close_13,close_21,regular_bearish,hidden_bearish
0,72.789165,-0.177410,0.221228,2.092604,0.200951,1.001188,1.223042,1.801425,0,0
1,69.753067,-0.215517,0.197262,2.028301,0.568225,0.550264,1.038805,1.967740,0,0
2,69.711123,-0.266054,0.186139,1.880235,0.683244,0.972035,1.232139,1.361463,0,0
3,63.856753,-0.407092,0.168855,1.627332,0.704499,0.650897,1.351636,1.167684,0,0
4,56.515958,-0.634296,0.164395,1.350274,0.074355,0.870586,1.361457,0.811388,0,0
...,...,...,...,...,...,...,...,...,...,...
4619,35.196819,-0.474534,0.239216,0.816744,0.083638,0.253499,-0.041155,-0.824430,0,0
4620,43.554171,-0.431766,0.259344,0.785037,0.138295,0.397630,-0.271720,-0.764778,0,0
4621,43.161297,-0.388863,0.230933,0.811599,0.473288,0.368514,-0.153755,-0.489137,0,0
4622,38.906900,-0.399572,0.172141,0.871302,0.285269,0.265432,-0.254812,-0.787004,0,0


In [13]:
data1 = data1.drop(['Datetime', 'Open', 'Close', 'Volume'], axis=1)
data1

,rsi,macd,cmf,bb_width,close_3,close_8,close_13,close_21,regular_bearish,hidden_bearish
20,42.675249,-1.167245,-0.105120,2.696486,-0.413648,-0.513035,0.132982,0.353906,1,0
21,42.552947,-0.923524,-0.069361,2.610296,-0.538648,-0.520376,0.002016,0.461903,1,0
22,39.322868,-0.831740,-0.079294,2.533792,-0.841096,-0.534257,-0.562830,0.152422,1,0
23,48.796865,-0.450733,0.013955,2.375705,-0.179184,-0.013114,0.238808,0.983291,1,0
30,46.567826,0.358228,0.080721,1.524780,-0.028422,0.293775,0.683030,0.595567,0,1
...,...,...,...,...,...,...,...,...,...,...
4579,45.623839,-0.227830,-0.100382,0.667843,0.157983,0.336525,0.516383,0.324619,1,0
4586,38.580593,-0.517092,-0.083125,0.964843,-0.044624,0.136777,0.091522,-0.522602,0,1
4587,37.372356,-0.516589,0.043349,1.028683,0.180465,0.331741,-0.011946,-0.533442,0,1
4603,62.162710,0.515202,0.113407,0.800047,-0.000250,-0.037212,0.282216,0.659713,1,0


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor(random_state=1)

X = data1.drop(['close_3', 'close_8', 'close_13', 'close_21'], axis=1).to_numpy()
y = data1[['close_3', 'close_8', 'close_13', 'close_21']].to_numpy()

X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

(1369, 6)
(1369, 4)


In [16]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=1)

clf.fit(X_train, y_train)
prediction = clf.predict(X_test)

from sklearn.metrics import r2_score

print(metrics.r2_score(y_test, prediction, multioutput='uniform_average'))
print(metrics.r2_score(y_test, prediction, multioutput='raw_values'))

0.31535828487862294
[0.29899181 0.32330214 0.44485223 0.19428696]
